
1. Columns not tested yet:'Sales Username','Charge Date'

In [75]:
import snowflake.connector
from get_secret import get_secret
from snowflake.connector.pandas_tools import pd_writer
import json
import pandas as pd

# Import and clean snowflake views

In [ ]:
# Set up the Snowflake connection parameters
config_json = json.loads(get_secret())
config_json.update({'warehouse':'COMPUTE_WH', 'database':'STAGE_DEV','schema': 'AMELIA_DBO',"loglevel":'DEBUG'})

# Create a connection object
conn = snowflake.connector.connect(**config_json)
cur = conn.cursor()

sql_file_path = 'Revenue/Amelia_Revenue_RAW_Data.sql'
sql_script = read_sql_file(sql_file_path)
cur.execute(sql_script)

result_df = cur.fetch_pandas_all()

In [61]:
snowflake_pd_df = result_df

In [62]:
snowflake_pd_df['\tPercent of Full Leg'].unique()

array(['100%'], dtype=object)

In [63]:
columns_to_remove = ['\tPercent of Full Leg','Charge Date','Sales Username']
snowflake_pd_df = snowflake_pd_df.drop(columns=columns_to_remove)

snowflake_pd_df['Flight Date'] = pd.to_datetime(snowflake_pd_df['Flight Date'])
snowflake_pd_df['Net Charge'] = snowflake_pd_df['Net Charge'].astype(float)
snowflake_pd_df['Taxes'] = snowflake_pd_df['Taxes'].astype(float)
snowflake_pd_df['Total Charge'] = snowflake_pd_df['Total Charge'].astype(float)
snowflake_pd_df['Flight Nmbr'] = snowflake_pd_df['Flight Nmbr'].astype(int)
#snowflake_pd_df['Charge Date'] = pd.to_datetime(snowflake_pd_df['Charge Date']).dt.tz_localize('UTC').dt.tz_convert('US/Mountain').dt.strftime('%m/%d/%Y')

# Import and clean report data

In [64]:
report_df = pd.read_csv('Revenue/Revenue_Raw_Data_0910_Beta.csv', skiprows=0)   

In [66]:
columns_to_remove = [ 'LastMod_SalesUser','Source','ActualFlightDate','mny_Tax_1_Percentage', "Percent",'ChargeDate', 'Base_Charge','Base_Taxes','TotalCharge1','mny_Exchange_Rate','PositiveChargePAX','CancellationDate',
       'PaxStatus', 'ChargeStatus', 'SeatSold']
report_df = report_df.drop(columns=columns_to_remove)

report_df['NetCharge'] = report_df['NetCharge'].str.replace('$', '')
report_df['NetCharge'] = report_df['NetCharge'].str.replace(')', '')
report_df['NetCharge'] = report_df['NetCharge'].str.replace('(', '-')
report_df['NetCharge'] = report_df['NetCharge'].astype(float)

report_df['Taxes'] = report_df['Taxes'].str.replace('$', '')
report_df['Taxes'] = report_df['Taxes'].str.replace(')', '')
report_df['Taxes'] = report_df['Taxes'].str.replace('(', '-')
report_df['Taxes'] = report_df['Taxes'].astype(float)

report_df['TotalCharge'] = report_df['TotalCharge'].str.replace('$', '')
report_df['TotalCharge'] = report_df['TotalCharge'].str.replace(')', '')
report_df['TotalCharge'] = report_df['TotalCharge'].str.replace('(', '-')
report_df['TotalCharge'] = report_df['TotalCharge'].astype(float)

#report_df['Percent'] = report_df['Percent'].str.replace('%', '')
#report_df['mny_Exchange_Rate'] = report_df['mny_Exchange_Rate'].astype(float)
report_df['FlightDate1'] = pd.to_datetime(report_df['FlightDate1'])
report_df['FlightDate1'] = report_df['FlightDate1'].dt.strftime('%m/%d/%Y')
report_df['FlightDate1'] = pd.to_datetime(report_df['FlightDate1'])


# Compare two dataframes

In [67]:
report_df.columns = snowflake_pd_df.columns 

In [72]:
def calculate_df2_accuracy(df1, df2):
    merged = df1.merge(df2, how = 'outer', indicator = True)
    df1_only = merged[merged['_merge'] == "left_only"]
    df2_only = merged[merged['_merge'] == "right_only"]


    # Calculate accuracy as a percentage
    accuracy_df1 = ((len(df1) - len(df1_only)) / len(df1)) * 100
    accuracy_df2 = ((len(df2) - len(df2_only)) / len(df2)) * 100

    return accuracy_df1,accuracy_df2,df1_only,df2_only


In [73]:
report_accuracy,snowflake_accuracy,report_only,snowflake_only = calculate_df2_accuracy(report_df,snowflake_pd_df)
print(f"Accuracy: {snowflake_accuracy:.2f}%" " snowflake records in report\nThe number of records in snowflake but not in report is", len(snowflake_only))
print(f"Accuracy: {report_accuracy:.2f}%"" report records in snowflake\nThe number of records in report but not in snowflake is", len(report_only))

Accuracy: 94.73% snowflake records in report
The number of records in snowflake but not in report is 4406
Accuracy: 94.73% report records in snowflake
The number of records in report but not in snowflake is 4365


# Output the difference file

In [74]:
try:
    snowflake_only.to_csv("Revenue/snowflake_only_record.csv", index=False) 
except:
    print("No snowflake only records")
try:
    report_only.to_csv("Revenue/report_only_record.csv", index=False) 
except:
    print("No report only records")